In [1]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
pip install fastembed

Note: you may need to restart the kernel to use updated packages.


In [25]:
from fastembed import TextEmbedding

model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_handle)

In [26]:
#Q1:

# Your query to embed
q1 = ["I just discovered the course. Can I join now?"]

# Generate embeddings (returns a generator)
embeddings_generator_q1 = embedding_model.embed(q1)

# Convert generator to list (each element is a numpy array)
embeded_query_q1 = list(embeddings_generator_q1)[0]

# embeddings_list[0] is the numpy array embedding for the query
print("Embeded vector shape:", embeded_query_q1.shape)

# Minimal value in the embedding vector
print("Minimal value in embeded vector:", embeded_query_q1.min())

Embeded vector shape: (512,)
Minimal value in embeded vector: -0.11726373885183883


In [11]:
#Q2
doc = ['Can I still join the course after the start date?']
embeddings_generator_q2 = embedding_model.embed(doc)
embeded_query_q2 = list(embeddings_generator_q2)[0]

embeded_query_q2.dot(embeded_query_q1)
#embeded_query_q1.dot(embeded_query_q2)

np.float64(0.9008528895674548)

In [15]:
from qdrant_client import QdrantClient, models

In [16]:
qd_client = QdrantClient("http://localhost:6333")

In [33]:
#Q3 Begin
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [14]:
documents[2]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [70]:
collection_name = "homework-02-Q3"

qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [30]:
points = []

for i, doc in enumerate(documents):
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload=doc
    )
    points.append(point)


In [32]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [56]:
def search(query, limit=5):

    query_points  = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        limit=limit,
        with_payload=True #to get metadata in the results
    )

    return query_points

In [57]:
query_points = search("I just discovered the course. Can I join now?")

In [66]:
#Q3 End
for point in query_points.points:
    print(point.id, point.score)


1 0.8182378
2 0.80853975
0 0.7629685
4 0.7304499
3 0.71330786


In [68]:
#Q4 Begin
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=text, model=model_handle),
        payload=doc
    )
    points.append(point)

In [71]:
collection_name = "homework-02-Q4"

qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [72]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [73]:
def search(query, limit=5):

    query_points  = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        limit=limit,
        with_payload=True #to get metadata in the results
    )

    return query_points

In [74]:
query_points = search("I just discovered the course. Can I join now?")

In [76]:
#Q4 End
for point in query_points.points:
    print(point.id, point.score)

0 0.8514543
1 0.8436594
2 0.84082866
4 0.8086008
3 0.7755158


In [82]:
#Q5
# Initialize variables to track the model with smallest dimension
min_dim = 512
min_dim_model = ''

# Iterate through all supported models
for model in TextEmbedding.list_supported_models():
   
    # Check if current model has smaller dimension
    if model["dim"] < min_dim:
        min_dim = model["dim"]
        min_dim_model = model["model"]


print(f"Model with smallest dimension: {min_dim_model} ({min_dim} dimensions)")

Model with smallest dimension: BAAI/bge-small-en (384 dimensions)


In [83]:
#Q6 Begin
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [89]:
model_handle = "BAAI/bge-small-en"

In [84]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=text, model=model_handle),
        payload=doc
    )
    points.append(point)

In [87]:
collection_name = "homework-02-Q6"

EMBEDDING_DIMENSIONALITY = 384

qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [88]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.03it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [90]:
def search(query, limit=1):

    query_points  = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        limit=limit,
        with_payload=True #to get metadata in the results
    )

    return query_points

In [91]:
query_points = search("I just discovered the course. Can I join now?")

In [93]:
#Q6 End
query_points

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])